In [ ]:
#456!apt-get update
#!apt-get install -y libgl1-mesa-glx libsm6 libxext6 libxrender-dev
!pip install opencv-python
!pip install torch
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.1/874.1 kB 17.6 MB/s eta 0:00:00


In [ ]:
import pickle
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import sys
import math
import torch.nn as nn
import torch.nn.init as init
import matplotlib.pyplot as plt

from ultralytics import YOLO
from torch.utils.data import TensorDataset, DataLoader, random_split

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# pool of square window of size=3, stride=2
import torch
m = torch.nn.MaxPool2d(2, stride=2)
# pool of non-square window
#m = torch.nn.MaxPool2d((3, 2), stride=(2, 1))
input = torch.randn(10, 3, 32, 32)
conv2d_1 = torch.nn.Conv2d(3, 64, kernel_size=3, padding=1)
conv2d_2 = torch.nn.Conv2d(64, 64, kernel_size=3, padding=1)

conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3, padding=1)
conv2d_4 = torch.nn.Conv2d(128, 128, kernel_size=3, padding=1)

conv2d_5 = torch.nn.Conv2d(128, 256, kernel_size=3, padding=1)
conv2d_6 = torch.nn.Conv2d(256, 256, kernel_size=3, padding=1)

conv2d_7 = torch.nn.Conv2d(256, 512, kernel_size=3, padding=1)
conv2d_8 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)

conv2d_9 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
conv2d_10 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)

block1 = [ conv2d_1, torch.nn.ReLU(inplace=True), conv2d_2, torch.nn.ReLU(inplace=True), m ]
block2 = [ conv2d_3, torch.nn.ReLU(inplace=True), conv2d_4, torch.nn.ReLU(inplace=True), m ]
block3 = [ conv2d_5, torch.nn.ReLU(inplace=True), conv2d_6, torch.nn.ReLU(inplace=True), m ]
block4 = [ conv2d_7, torch.nn.ReLU(inplace=True), conv2d_8, torch.nn.ReLU(inplace=True), m ]
block5 = [ conv2d_9, torch.nn.ReLU(inplace=True), conv2d_10, torch.nn.ReLU(inplace=True), m ]
layer1 = torch.nn.Sequential(*block1)
layer2 = torch.nn.Sequential(*block2)
layer3 = torch.nn.Sequential(*block3)
layer4 = torch.nn.Sequential(*block4)
layer5 = torch.nn.Sequential(*block5)

layers = [layer1, layer2, layer3, layer4, layer5]
output = []
tmpInput = input
for layerIdx, layer in enumerate(layers):
  output.append(layer(tmpInput))
  tmpInput = output[layerIdx]

output_m1 = torch.cat((m(output[1]), output[2]), 1)
output_m2 = torch.cat((m(output_m1), output[3]), 1)
output_m3 = torch.cat((m(output_m2), output[4]), 1)
#ouput_pre_classification = output_m2.view(output_m2.size(0), -1)
ouput_pre_classification = output_m3.view(output_m3.size(0), -1)

#output1 = layer1(input)
#output2 = layer2(output1)
#output3 = layer3(output2)
#output4 = layer4(output3)

# output_m1 = torch.cat((m(output2), output3), 1)
# output_m2 = torch.cat((m(output_m1), output4), 1)

print(input.size())
print(ouput_pre_classification.size())
print(output[-1].size())


torch.Size([10, 3, 32, 32])
torch.Size([10, 1408])
torch.Size([10, 512, 1, 1])


##**Utils**

In [ ]:
label_name_mapping = {
    0: "3-junction",
    1: "One-way road",
    2: "Side to obe followed",
    3: "Cross road",
    4: "Intersection with Uncontrolled Road",
    5: "Dangerous turn",
    6: "No Left turn",
    7: "Bus stop",
    8: "Roundabout",
    9: "No parking and stopping",
    10: "U-turn",
    11: "Lane-allocation",
    12: "No left turn for motorcycles",
    13: "Slow Down",
    14: "No Trucks Allowed",
    15: "Narrow Road on the Right",
    16: "No Passenger Cars and Trucks",
    17: "Height Limit",
    18: "No U-Turn",
    19: "No U-Turn and No Right Turn",
    20: "No Cars Allowed",
    21: "Narrow Road on the Left",
    22: "Uneven Road",
    23: "No Two or Three-wheeled Vehicles",
    24: "Customs Checkpoint",
    25: "Motorcycles Only",
    26: "Obstacle on the Road",
    27: "Children Presence",
    28: "Trucks and Containers",
    29: "No Motorcycles Allowed",
    30: "Trucks Only",
    31: "Road with Surveillance Camera",
    32: "No Right Turn",
    33: "Series of Dangerous Turns",
    34: "No Containers Allowed",
    35: "No Left or Right Turn",
    36: "No Straight and Right Turn",
    37: "Intersection with T-Junction",
    38: "Speed limit (50km/h)",
    39: "Speed limit (60km/h)",
    40: "Speed limit (80km/h)",
    41: "Speed limit (40km/h)",
    42: "Left Turn",
    43: "Low Clearance",
    44: "Other Danger",
    45: "Go Straight",
    46: "No Parking",
    47: "No Left or U-turn",
    48: "No U-Turn for Cars",
    49: "Level Crossing with Barriers"
}

def display_batch(images, labels):
    num_images = len(images)
    plt.figure(figsize=(5, 5 * num_images))  # Adjust the figure size to have more space for each image

    for i in range(num_images):
        plt.subplot(num_images, 1, i + 1)
        plt.imshow(images[i].permute(1, 2, 0))  # Convert from CxHxW to HxWxC for plt.imshow
        plt.title(label_name_mapping[labels[0][i].item()])
        plt.axis('off')  # Hide axes for better readability

    plt.tight_layout()  # Adjust subplots to fit into the figure area.
    plt.show()

def pickle_data(file, writeColumns=None):
    """
    Read/Write pickle training/testing data, models to avoid
    loading data again (time consuming)

    :param file: path to pickle file
    :param writeColumns (torch.Tensor or np.ndarray): variables to be saved to pickle file

    :returns :
    If writeColumns = None -> tuple(torch.Tensor)
    tuple()
    """
    if writeColumns is None:
        with open(file, mode="rb") as f:
            dataset = pickle.load(f)
            return tuple( # Convert the pickled data into tensor if it is of any other types
                map(lambda col: torch.tensor(dataset[col])
                    if not type(dataset[col]) == torch.Tensor else dataset[col],
                    ['images', 'labels'])
                )
                # lambda(col) where columns are the inputs
    else:
        with open(file, mode="wb") as f:
            dataset = pickle.dump({"images": writeColumns[0], "labels": writeColumns[1]}, f)
            print("Data is saved in", file)

    def preprocess_batch(batch_tensor):
      preprocessed_image = []
      idx = 0
      for img_tensor in batch_tensor:
         # Convert tensor to numpy array
        #print("{}: {}".format(idx, img_tensor.size()))
        img_np = img_tensor.permute(1, 2, 0).numpy()  # [C, H, W] -> [H, W, C]

         # Convert from [0, 1] or [0, 255] to [0, 255] and ensure uint8 type
        if img_np.max() <= 1.0:
            img_np = (img_np * 255).astype(np.uint8)
        else:
            img_np = img_np.astype(np.uint8)

        img_res = preprocess(img_np)
        img_tensor = torch.from_numpy(img_res).unsqueeze(0)
        preprocessed_image.append(img_tensor)
        idx += 1

      preprocessed_tensor = torch.stack(preprocessed_image)
      return preprocessed_tensor

def preprocess(image):
    if image.shape[2] > 1:
      image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      # Noise reduction
      image = cv2.GaussianBlur(image, (3, 3), 1,5)
      # Histogram equalization
      image = cv2.equalizeHist(image)
      # Image eroding
      image = cv2.erode(image, (3, 3))
      # Resize image
      image = cv2.resize(image, (32, 32))
    return image

class CustomDataset(TensorDataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

# Germany
# train_images, train_labels  = pickle_data(file = '/content/drive/MyDrive/TrafficSignData/German/gray32_train_dataset')
# val_images, val_labels      = pickle_data(file = '/content/drive/MyDrive/TrafficSignData/German/gray32_val_dataset')

# Vietnam
vietnam_imgs, vietnam_labels = pickle_data(file = '/content/drive/MyDrive/TrafficSignData/Vietnam/gray_processed_dataset')
portion    = 0.8
train_size = int(portion * len(vietnam_imgs))
val_size   = len(vietnam_imgs) - train_size
train_dataset, val_dataset = random_split(list(zip(vietnam_imgs, vietnam_labels)), [train_size, val_size])
print(len(val_dataset))

2919


##**Object detection**

In [ ]:
ROOT_DIR = '/content/drive/MyDrive/TrafficSignData/Vietnam/Object Detection'
model = YOLO('yolov5s.yaml')
results = model.train(data=os.path.join(ROOT_DIR, 'obj_detection.yaml'), epochs=30)

In [ ]:
!apt-get install -y locales
!locale-gen en_US.UTF-8
!update-locale LANG=en_US.UTF-8

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
!mv /content/runs /content/drive/MyDrive/TrafficSignData/Vietnam/Object Detection/Model_240824

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

## **Network**


In [ ]:
#%%writefile my_mainScript.py

# Ensure that the input tensor size is [batch_size, channel, width, heigh]
# Ensure that the labels are int64
# Ensure that the ouput of the classfier matches the number of classes

supportedArch = [
    'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]

class VGG(nn.Module):
    '''
    VGG model
    '''
    def __init__(self, vgg_blocks):
        super(VGG, self).__init__()
        self.layers = nn.ModuleList(vgg_blocks)
        self.maxPool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(1408, 512), # nn.Linear(1408, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 50),
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()

    def forward(self, x):
      output = []
      tmpInput = x
      for layerIdx, layer in enumerate(self.layers):
        current_output = layer(tmpInput)
        output.append(current_output)
        tmpInput = current_output

      output_m1 = torch.cat((self.maxPool (output[1]), output[2]), 1)
      output_m2 = torch.cat((self.maxPool (output_m1), output[3]), 1)
      output_m3 = torch.cat((self.maxPool (output_m2), output[4]), 1)

      ouput_pre_classification = output_m3.view(output_m3.size(0), -1)
      #ouput_pre_classification = output[-1].view(output[-1].size(0), -1)
      #print(ouput_pre_classification.size())
      final_ouput = self.classifier(ouput_pre_classification)
      return final_ouput

    def vgg11():
        """VGG 11-layer model (configuration "A")"""
        return VGG(make_layers(cfg['A']))


    def vgg11_bn():
        """VGG 11-layer model (configuration "A") with batch normalization"""
        return VGG(make_layers(cfg['A'], batch_norm=True))


    def vgg13():
        """VGG 13-layer model (configuration "B")"""
        return VGG(make_layers(cfg['B']))


    def vgg13_bn():
        """VGG 13-layer model (configuration "B") with batch normalization"""
        return VGG(make_layers(cfg['B'], batch_norm=True))


    def vgg16():
        """VGG 16-layer model (configuration "D")"""
        return VGG(make_layers(cfg['D']))


    def vgg16_bn():
        """VGG 16-layer model (configuration "D") with batch normalization"""
        return VGG(make_layers(cfg['D'], batch_norm=True))


    def vgg19():
        """VGG 19-layer model (configuration "E")"""
        return VGG(make_layers(cfg['E']))


    def vgg19_bn():
        """VGG 19-layer model (configuration 'E') with batch normalization"""
        return VGG(make_layers(cfg['E'], batch_norm=True))

def make_layers(cfg, batch_norm=False):
    layers = []
    vgg_blocks = []
    in_channels = 1
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            vgg_sequential = nn.Sequential(*layers)
            vgg_blocks.append(vgg_sequential)
            layers = [] # empty the current block
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return vgg_blocks

cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M',
          512, 512, 512, 512, 'M'],
}

model_names = sorted(name for name in supportedArch
    if name.islower() and not name.startswith("__")
                     and name.startswith("vgg")
                    )

best_prec1     = 0
ARCH           = "vgg16_bn"
WORKERS        = 4
EPOCHS         = 100
START_EPOCH    = 0
BATCH_SIZE     = 32
LEARNING_RATE  = 0.05
MOMENTUM       = 0.9
WEIGHT_DECAY   = 5E-4
PRINT_FREQ     = 20
RESUME         = "/content/drive/MyDrive/TrafficSignData/Vietnam/24.08.18_Take1/checkpoint_99.tar" # Path to model file
EVALUATE       = True
HALF_PRECISION = False
CPU            = True
SAVE_DIR       = "save_temp"

def main():
    global best_prec1, START_EPOCH
    print (VGG.__dict__)

    # Check the save_dir exists or not
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

    # model = VGG.__dict__[args.arch]()
    model_method = getattr(VGG, ARCH)
    model = model_method()
    #model.features = torch.nn.DataParallel(model.features)
    if CPU:
        model.cpu()
    else:
        model.cuda()

    # optionally resume from a checkpoint
    if RESUME:
        if os.path.isfile(RESUME):
            print("=> loading checkpoint '{}'".format(RESUME))
            if not CPU:
              checkpoint = torch.load(RESUME)
            else:
              checkpoint = torch.load(RESUME, map_location=torch.device('cpu'))
            START_EPOCH = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(EVALUATE, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(RESUME))

    cudnn.benchmark = True

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

    # define loss function (criterion) and pptimizer
    criterion = nn.CrossEntropyLoss()
    if CPU:
        criterion = criterion.cpu()
    else:
        criterion = criterion.cuda()

    if HALF_PRECISION:
        model.half()
        criterion.half()

    optimizer = torch.optim.SGD(model.parameters(), LEARNING_RATE,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

    if EVALUATE:
        validate(val_loader, model, criterion)
        return

    for epoch in range(START_EPOCH, EPOCHS):
        adjust_learning_rate(optimizer, epoch)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)

        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
        }, is_best, filename=os.path.join(SAVE_DIR, 'checkpoint_{}.tar'.format(epoch)))


def train(train_loader, model, criterion, optimizer, epoch):
    """
        Run one train epoch
    """
    batch_time = AverageMeter()
    data_time  = AverageMeter()
    losses     = AverageMeter()
    top1       = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        target = target.type(torch.int64)
        # measure data loading time
        data_time.update(time.time() - end)

        if CPU == False:
            input = input.cuda(non_blocking=True)
            target = target.cuda(non_blocking=True)
        if HALF_PRECISION:
            input = input.half()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))


def validate(val_loader, model, criterion):
    """
    Run evaluation
    """
    batch_time = AverageMeter()
    losses     = AverageMeter()
    top1       = AverageMeter()
    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
      # TODO: preprocess here
        target = target.type(torch.int64)
        if CPU == False:
            input = input.cuda(non_blocking = True)
            target = target.cuda(non_blocking = True)

        if HALF_PRECISION:
            input = input.half()

        # compute output
        with torch.no_grad():
            output = model(input)
            loss = criterion(output, target)

        output = output.float()
        #print(target.size())
        loss = loss.float()

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # if (i == 0):
        #   maxk = max((1,))
        #   batch_size = target.size(0)

        #   _, output_lbl = output.topk(maxk, 1, True, True)
        #   output_lbl = output_lbl.t()
        #   display_batch(input, output_lbl)
        if i % PRINT_FREQ == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(val_loader), batch_time=batch_time, loss=losses,
                      top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))

    return top1.avg

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    """
    Save the training model
    """
    torch.save(state, filename)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 2 every 30 epochs"""
    lr = LEARNING_RATE * (0.5 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    # Print predictions and targets for debugging
    # print("Predictions (top-k indices):")
    # print(pred)
    # print("True Labels:")
    # print(target)

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


if __name__ == '__main__':
    main()

{'__module__': '__main__', '__doc__': '\n    VGG model\n    ', '__init__': <function VGG.__init__ at 0x7f7b4d4d2560>, 'forward': <function VGG.forward at 0x7f7b4d4d25f0>, 'vgg11': <function VGG.vgg11 at 0x7f7b4d4d2680>, 'vgg11_bn': <function VGG.vgg11_bn at 0x7f7b4d4d2710>, 'vgg13': <function VGG.vgg13 at 0x7f7b4d4d27a0>, 'vgg13_bn': <function VGG.vgg13_bn at 0x7f7b4d4d2830>, 'vgg16': <function VGG.vgg16 at 0x7f7b4d4d28c0>, 'vgg16_bn': <function VGG.vgg16_bn at 0x7f7b4d4d2950>, 'vgg19': <function VGG.vgg19 at 0x7f7b4d4d29e0>, 'vgg19_bn': <function VGG.vgg19_bn at 0x7f7b4d4d2a70>}
=> loading checkpoint '/content/drive/MyDrive/TrafficSignData/Vietnam/24.08.18_Take1/checkpoint_99.tar'
=> loaded checkpoint 'True' (epoch 100)
Test: [0/92]	Time 0.249 (0.249)	Loss 0.0244 (0.0244)	Prec@1 100.000 (100.000)
Test: [20/92]	Time 0.245 (0.249)	Loss 0.0001 (0.0286)	Prec@1 100.000 (99.851)
Test: [40/92]	Time 0.350 (0.258)	Loss 0.0557 (0.0369)	Prec@1 96.875 (99.314)
Test: [60/92]	Time 0.246 (0.273)	Los

In [ ]:
!python my_mainScript.py --arch vgg16_bn --batch-size 64

In [ ]:
import os
import cv2
import torch
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader

IMAGE_EXTENSIONS = {'.png', '.jpg', '.jpeg', '.bmp'}
IMG_WIDTH  = 32
IMG_HEIGHT = 32

class ImageDataset(Dataset):
    def __init__(self,
                 folder_path=None, # input: image folder to be converted to dataset
                 images=None,      # input: image tensor [num_imgs, C, W, H]
                 labels=None,      # input: labels tensor [num_labels]
                 transform=None    # input: transformation method applied to images
                 ):
        self.transform = transform

        if folder_path:
            # Loading images from folder
            self.image_files = [f for f in os.listdir(folder_path) if os.path.splitext(f)[1].lower() in IMAGE_EXTENSIONS]
            self.folder_path = folder_path
            self.images = None
            self.labels = None
        elif images is not None and labels is not None:
            # Using pre-loaded tensors
            self.images = images
            self.labels = labels
            self.image_files = None
            self.folder_path = None
        else:
            raise ValueError("Either folder_path or (images, labels) must be provided")

    def __len__(self):
        if self.image_files:
            return len(self.image_files)
        elif self.images is not None:
            return len(self.images)
        else:
            raise RuntimeError("Dataset is not properly initialized")

    def __getitem__(self, idx):
        if self.image_files:
            # Load image from disk
            image_file = self.image_files[idx]
            image_path = os.path.join(self.folder_path, image_file)
            image = Image.open(image_path).convert('RGB')
        else:
            # Get image from pre-loaded tensors
            image = self.images[idx]

        # Apply transformation to image
        if self.transform:
            image = self.transform(image)

        # Label tensor is available, add it to the dataset also
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        else:
            return image

class PreProcessing:
    LOWER_THRESH = 50
    UPPER_THRESH = 200

    def __init__(self):
        pass

    """
        Input images are expected to be colored
    """
    def augment_data(self,
                     input_folder_path,
                     output_folder_path,
                     target_num_data):

        # Define transformations for augmentation
        augmentation_transforms = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(20),
            transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
        ])

        # Load images into a tensor
        dataset    = ImageDataset(input_folder_path, transform=transforms.ToTensor())
        dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

        # Ensure output folder exists
        os.makedirs(output_folder_path, exist_ok=True)

        images = []

        # Collect all images in a list
        for images_batch in dataloader:
            images.append(images_batch[0])

        images = torch.stack(images)

        # Shuffle the images
        images = images[torch.randperm(images.size(0))]

        # Apply augmentations until target number of images is reached
        org_img_idx  = 0
        num_org_imgs = len(dataset)
        augmented_count = target_num_data - len(dataset)
        for augIdx in range(augmented_count):
            img = images[org_img_idx]
            img = transforms.ToPILImage()(img)
            augmented_image = augmentation_transforms(img)
            augmented_image_path = os.path.join(output_folder_path, f'augmented_{augIdx + 1}.png')
            augmented_image.save(augmented_image_path)

            org_img_idx = (org_img_idx + 1) % num_org_imgs

    def apply_preprocessing(self,
                            input_folder_path,
                            output_folder_path):
        self.create_folder(self, output_folder_path)

        for filename in os.listdir(input_folder_path):
            # Get the file extension
            _, ext = os.path.splitext(filename)

            # Check if the file is an image
            if ext.lower() in IMAGE_EXTENSIONS:
                file_path = os.path.join(input_folder_path, filename)
                image     = cv2.imread(file_path)

                # Convert to grayscale
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                # Noise reduction
                image = cv2.GaussianBlur(image, (3, 3), 1.5)

                # Otsu threshold
                _, image = cv2.threshold(
                    image,
                    0,
                    255, # images at exceeding the threshold is set to 255
                    cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                # Histogram equalization
                image = cv2.equalizeHist(image)

                # Image eroding
                image = cv2.erode(image, (3, 3))

                # Resize image
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))

                output_filepath = os.path.join(output_folder_path, filename)
                cv2.imwrite(output_filepath, image)

    @staticmethod
    def create_folder(self, folder_path):
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    @staticmethod
    def save_images(self, dataset: Dataset, output_folder_path: str):
        os.makedirs(output_folder_path, exist_ok=True)

        dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
        for idx, (image_batch, _) in enumerate(dataloader):
            img = image_batch[0]
            img_pil = transforms.ToPILImage()(img)
            original_image_path = os.path.join(output_folder_path, f'original_{idx + 1}.png')
            img_pil.save(original_image_path)


if __name__ == '__main__':
    INPUT_DIR                = r"C:\MSPACE\02_DAT\TestImg"
    OUTPUT_DIR_PREPROCESSING = r"C:\MSPACE\02_DAT\output_process"
    OUTPUT_DIR_AUGMENTATION  = r"C:\MSPACE\02_DAT\output_augmentation"
    preprocessor = PreProcessing()

    #preprocessor.apply_preprocessing(INPUT_DIR, OUTPUT_DIR_PREPROCESSING)

    """
        ImageDataSet usage:

        1. Image folder:
            transform = transform.ToTensor()
            dataset_from_folder    = ImageDataset(folder_path='path_to_images', transform=transform)
            dataloader_from_folder = DataLoader(dataset_from_folder, batch_size=10, shuffle=True)

        2. Image and Label tensors:
            images = torch.randn(100, 3, 128, 128)  # Example tensor with shape (num_samples, channels, height, width)
            labels = torch.randint(0, 10, (100,))   # Example tensor with class labels
            dataset_from_tensors    = ImageDataset(images=images, labels=labels, transform=transform)
            dataloader_from_tensors = DataLoader(dataset_from_tensors, batch_size=10, shuffle=True)
    """

    preprocessor.augment_data(INPUT_DIR, OUTPUT_DIR_AUGMENTATION, 5)
    # image_index = 0

    # for i in range(20):
    #     # Move to the next image, wrap around if needed
    #     print(image_index)
    #     image_index = (image_index + 1) % 3

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\MSPACE\\02_DAT\\TestImg'